In [1]:
!pip install neattext
%pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer

In [5]:
pip install scikit-learn as sklearn

ERROR: Could not find a version that satisfies the requirement as (from versions: none)
ERROR: No matching distribution found for as


In [6]:
pip install plotly-express

   ---------------------------------------- 0.0/19.1 MB ? eta -:--:--
   - -------------------------------------- 0.8/19.1 MB 8.3 MB/s eta 0:00:03
   ---- ----------------------------------- 2.4/19.1 MB 6.4 MB/s eta 0:00:03
   --------- ------------------------------ 4.7/19.1 MB 8.1 MB/s eta 0:00:02
   -------------- ------------------------- 6.8/19.1 MB 8.6 MB/s eta 0:00:02
   ------------------ --------------------- 8.9/19.1 MB 8.9 MB/s eta 0:00:02
   ----------------------- ---------------- 11.0/19.1 MB 9.0 MB/s eta 0:00:01
   ---------------------------- ----------- 13.4/19.1 MB 9.3 MB/s eta 0:00:01
   --------------------------------- ------ 15.7/19.1 MB 9.5 MB/s eta 0:00:01
   ------------------------------------- -- 18.1/19.1 MB 9.8 MB/s eta 0:00:01
   ---------------------------------------- 19.1/19.1 MB 9.5 MB/s eta 0:00:00


In [7]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [8]:
import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split

import neattext.functions as nfx
import plotly.express as plx
from sklearn.metrics import classification_report
import keras
from keras.layers import Embedding,Dense,LSTM,GlobalMaxPooling1D,Input
from keras.callbacks import EarlyStopping,ReduceLROnPlateau
from keras.models import Sequential
import tensorflow as tf
from keras.utils import pad_sequences
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

In [13]:
data=pd.read_csv('Suicide_Detection.csv')
data.head()
# import os
# print([os.path.abspath(f) for f in os.listdir() if os.path.isfile(f)])

['C:\\Users\\KIIT0001\\Desktop\\icdcit\\suicide-and-depression-prediction.ipynb', 'C:\\Users\\KIIT0001\\Desktop\\icdcit\\suicide_depression_model.pkl', 'C:\\Users\\KIIT0001\\Desktop\\icdcit\\Suicide_Detection.csv']


In [16]:
# data=pd.read_csv('C:\Users\KIIT0001\suicide-watch.zip')
# data.head()

print([os.path.abspath(f) for f in os.listdir() if os.path.isfile(f)])

['C:\\Users\\KIIT0001\\Desktop\\icdcit\\suicide-and-depression-prediction.ipynb', 'C:\\Users\\KIIT0001\\Desktop\\icdcit\\suicide_depression_model.pkl', 'C:\\Users\\KIIT0001\\Desktop\\icdcit\\Suicide_Detection.csv']


In [17]:
data['class'].value_counts()

class
suicide        116037
non-suicide    116037
Name: count, dtype: int64

In [18]:
data['class'].value_counts().index.values

array(['suicide', 'non-suicide'], dtype=object)

In [19]:
train_data,test_data=train_test_split(data,test_size=0.2,random_state=10)

In [20]:
train_data['class'].value_counts().index.values

array(['suicide', 'non-suicide'], dtype=object)

In [21]:
def clean_text(text):
    text_length=[]
    cleaned_text=[]
    for sent in tqdm(text):
        sent=sent.lower()
        sent=nfx.remove_special_characters(sent)
        sent=nfx.remove_stopwords(sent)
        text_length.append(len(sent.split()))
        cleaned_text.append(sent)
    return cleaned_text,text_length

In [22]:
cleaned_train_text,train_text_length=clean_text(train_data.text)
cleaned_test_text,test_text_length=clean_text(test_data.text)

100%|██████████████████████████████████████████████████████████████████████████| 46415/46415 [00:07<00:00, 6130.01it/s]


In [ ]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(cleaned_train_text)

In [ ]:
cleaned_train_text

In [ ]:
train_text_seq=tokenizer.texts_to_sequences(cleaned_train_text)
train_text_pad=pad_sequences(train_text_seq,maxlen=50)


test_text_seq=tokenizer.texts_to_sequences(cleaned_test_text)
test_text_pad=pad_sequences(test_text_seq,maxlen=50)

In [ ]:
train_text_pad

Glove embeddings

In [ ]:
lbl_target=LabelEncoder()
train_output=lbl_target.fit_transform(train_data['class'])
test_output=lbl_target.transform(test_data['class'])

In [ ]:
import kagglehub
pickled_glove840b300d_for_10sec_loading_path = kagglehub.dataset_download('authman/pickled-glove840b300d-for-10sec-loading')

In [ ]:
import pickle
with open('/kaggle/input/pickled-glove840b300d-for-10sec-loading/glove.840B.300d.pkl', 'rb') as fp:
    glove_embedding = pickle.load(fp)

In [ ]:
v=len(tokenizer.word_index)

embedding_matrix=np.zeros((v+1,300), dtype=float)
for word,idx in tokenizer.word_index.items():
    embedding_vector=glove_embedding.get(word)
    if embedding_vector is not None:
        embedding_matrix[idx]=embedding_vector

In [ ]:
embedding_matrix

In [ ]:
early_stop=EarlyStopping(patience=5)
reducelr=ReduceLROnPlateau(patience=3)

keras sequential model

In [ ]:
model=Sequential()
model.add(Input(shape=(40,)))
model.add(Embedding(v+1,300,weights=[embedding_matrix],trainable=False))
model.add(LSTM(20,return_sequences=True))
model.add(GlobalMaxPooling1D())
model.add(Dense(256,activation='relu'))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer=keras.optimizers.SGD(0.1,momentum=0.09),loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
model.summary()

Model Training and Evaluation

In [ ]:
r=model.fit(train_text_pad,train_output,validation_data=(test_text_pad,test_output),
            epochs=20,batch_size=256,callbacks=[early_stop,reducelr])

In [ ]:
twt = ['i am happy']
twt = tokenizer.texts_to_sequences(twt)
twt = pad_sequences(twt, maxlen=50)

prediction = model.predict(twt)[0][0]
print(prediction)

if(prediction > 0.5):
    print("Potential Suicide Post")
else:
    print("Non Suicide Post")

In [ ]:
twt = ['Through these past years thoughts of suicide, fear, anxiety I’m so close to my limit']
twt = tokenizer.texts_to_sequences(twt)
twt = pad_sequences(twt, maxlen=50)


prediction = model.predict(twt)[0][0]
print(prediction)

if(prediction > 0.5):
    print("Potential Suicide Post")
elif (prediction == 1):
    print("Non Suicide Post")

In [ ]:
import pickle

# Assuming `model` is the trained model from the Kaggle notebook
# Replace this line with the actual training code or model object
# Example: model = RandomForestClassifier().fit(X_train, y_train)

# Save the model to a pickle file
pickle_file_path = 'suicide_depression_model.pkl'

# Serializing the model
with open(pickle_file_path, 'wb') as file:
    pickle.dump(model, file)

print(f"Model has been saved to {pickle_file_path}")

In [ ]:
from flask import Flask, request, jsonify
import pickle

# Load your trained NLP model (adjust the path to your pickle file)
with open('/kaggle/working/suicide_depression_model.pkl', 'rb') as f:
    model = pickle.load(f)

app = Flask(__name__)

# Define a route for predictions
@app.route('/predict', methods=['POST'])
def predict():
    try:
        # Parse input JSON
        data = request.get_json()

        # Ensure input is a string
        input_text = data.get('input_text')
        if not isinstance(input_text, str):
            return jsonify({'error': 'Invalid input. Expected a string.'}), 400

        # Predict using the model
        prediction = model.predict([input_text])  # Wrap input in a list for scikit-learn models

        # Return the result
        return jsonify({'prediction': prediction[0]})  # Adjust as needed for model output
    except Exception as e:
        return jsonify({'error': str(e)}), 500

# Ensure proper startup for Flask app
if __name__ == '__main__':
    # Use this block to avoid reloader issues
    app.run(host='0.0.0.0', port=5000, debug=False)

In [ ]:
!pip install flask-ngrok

In [ ]:
from flask_ngrok import run_with_ngrok

app = Flask(__name__)
run_with_ngrok(app)  # Starts ngrok when you run the Flask app

@app.route('/predict', methods=['POST'])
def predict():
    try:
        data = request.get_json()
        input_text = data.get('input_text')
        if not isinstance(input_text, str):
            return jsonify({'error': 'Invalid input. Expected a string.'}), 400
        prediction = model.predict([input_text])
        return jsonify({'prediction': prediction[0]})
    except Exception as e:
        return jsonify({'error': str(e)}), 500

if __name__ == '__main__':
    app.run()

In [ ]:
import requests
import json

url = "http://127.0.0.1:5000/predict"
headers = {"Content-Type": "application/json"}
data = {"input_text": "This is a test text"}

response = requests.post(url, headers=headers, data=json.dumps(data))
print(response.json())